# Семинар 11: декораторы и itertools

### Глава 1: декораторы

In [ ]:
def uppercase(func):  # передаем функцию, которую будем оборачивать
    def wrapper():
        result = func()
        return result.upper()
    return wrapper  # вернуть нужно функцию-обертку

In [ ]:
@uppercase
def print_hello():
    return "hello world"

In [ ]:
print_hello()

In [ ]:
class A:
    @uppercase
    def __str__(self):
        return 'some abstract class'

In [ ]:
a = A()
print(a)  # будет ли этот декоратор работать с классами? если нет, то как поправить?

При этом мы теряем метаданные о функции:

In [ ]:
A.__str__.__name__

In [ ]:
import functools


def uppercase(func):  # передаем функцию, которую будем оборачивать
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func()
        return result.upper()
    return wrapper  # вернуть нужно функцию-обертку

class A:
    @uppercase
    def __str__(self):
        return 'some abstract class'

In [ ]:
A.__str__.__name__

Для чего в реальной жизни удобно использовать декоратор?

Например, замерять суммарное время работы функций

In [ ]:
import functools
import time


def use_timer(result_struct: dict):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            elapsed_time = time.time() - start

            result_struct.setdefault(func.__name__, 0)
            result_struct[func.__name__] += elapsed_time

            return result
        
        return wrapper
    return decorator

class Worker:
    times = {}

    @use_timer(times)
    def do_light(self):
        for _ in range(10000):
            ...

    @use_timer(times)
    def do_medium(self):
        for _ in range(100000):
            ...

    @use_timer(times)
    def do_hard(self):
        for _ in range(1000000):
            ...

In [ ]:
w = Worker()

for _ in range(100):
    w.do_light()
    w.do_medium()
    w.do_hard()

In [ ]:
w.times

Декорировать можно также и классы

In [ ]:
from dataclasses import dataclass
from uuid import uuid4


def add_id(_class):
    class WithID(_class):
        def __init__(self, *args, **kargs):
            super().__init__(*args, **kargs)
            self.__id = str(uuid4())

        def get_id(self):
            return self.__id
    return WithID


@add_id
@dataclass
class Person:
    name: str
    email: str

In [ ]:
p = Person(name="Tema", email="astreltsov@hse.ru")
print(p.get_id())

Стоит учесть, что порядок декораторов важен!! Попробуйте поменять dataclass и add_id местами.

Можно еще подменять функции по ссылкам:

In [ ]:
from dataclasses import dataclass
from uuid import uuid4


def add_id(_class):
    original_init = _class.__init__

    def __init__(obj, *args, **kargs):
        original_init(obj, *args, **kargs)
        obj.id = str(uuid4())

    _class.__init__ = __init__
    return _class


@add_id
@dataclass
class Person:
    name: str
    email: str

In [ ]:
p = Person(name="Tema", email="astreltsov@hse.ru")
print(p.id)

### Глава 2: itertools

In [ ]:
a = [1, -1, 6, 3, -2, 0, -6]

list(filter(lambda x: x < 0, a))

In [ ]:
from functools import reduce

a = [1, 7, 3, 2, 5, 3]

reduce(lambda x, y: x + y, a)

In [ ]:
a = ["", "", "", "abc", "", "cde"]

reduce(lambda x, y: x or y, a)

In [ ]:
import itertools

In [ ]:
a = [1, 7, 3, 2, 5, 3]
list(itertools.accumulate(a))  # это как reduce с промежуточными значениями

In [ ]:
a = [1, 2, 3]
b = ["abc", "cde"]
c = [(1, 2), (3, 4)]

for x in itertools.chain(a, b, c):
    print(x)

In [ ]:
a = ["abc", "cde"]

for x in itertools.chain.from_iterable(a):
    print(x)

In [ ]:
a = "abcdef"

"".join(itertools.compress(a, [1, 0, 1, 1, 0, 0]))

In [ ]:
a = [-1, -2, -3, 6, 2, 1, -2, -1]

list(itertools.dropwhile(lambda x: x < 0, a))

In [ ]:
list(itertools.takewhile(lambda x: x < 0, a))

In [ ]:
a = [1, -1, 6, 3, -2, 0, -6]

list(itertools.filterfalse(lambda x: x < 0, a))

Пусть есть задача: надо сделать из строки вида "ABBBCC" строку вида "1A3B2C"

In [ ]:
s = input()

print(list(itertools.groupby(s)))

print(
    "".join(
        f"{len(list(group))}{symbol}"
        for symbol, group in itertools.groupby(s)
    ),
)

In [ ]:
a = [1, 2, 3, 4, 5, 6, 7]

for x in itertools.islice(a, 1, 5, 2):  # аналог среза только без создания массива нового
    print(x)

In [ ]:
a = ["A", "T", "G", "C"]

for x in itertools.pairwise(a):
    print(x)

In [ ]:
list(itertools.starmap(pow, [(2,5), (3,2), (10,3)])) #  func(*seq[0]), func(*seq[1]) и тд

In [ ]:
a = ["A", "T", "G", "C"]
b = [1, 2, 3, 4]

print(list(itertools.combinations(a, 2)))

In [ ]:
print(list(itertools.product(a, b)))

In [ ]:
print(list(itertools.permutations(a)))

In [ ]:
a = ["A", "T", "G", "C"]

iter = itertools.cycle(a)

print(next(iter))
print(next(iter))
print(next(iter))
print(next(iter))
print(next(iter))
print(next(iter))
print(next(iter))

### Зачем нужны itertools

1) Можно сделать свой спиннер

In [ ]:
import itertools
import sys
import time

def spinner(seconds):
    symbols = itertools.cycle('-|/')
    tend = time.time() + seconds
    while time.time() < tend:
        sys.stdout.write('\rPlease wait... ' + next(symbols)) # no newline
        sys.stdout.flush()
        time.sleep(0.1)
    print()

if __name__ == "__main__":
    spinner(3)

2) Быстро погруппировать словарь по значениям

In [ ]:
from operator import itemgetter


d = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": 1,
    "e": 2,
    "f": 3,
}

for value, items in itertools.groupby(sorted(d.items(), key=itemgetter(1)), key=itemgetter(1)):
    print(value, ":", *map(itemgetter(0), items))

3) Выбрать из списка чисел пару (тройку, четверку) с наибольшим произведением

In [ ]:
import functools

a = list(map(int, input().split()))
n = int(input())

print(
    max(
        itertools.combinations(a, n),
        key=functools.partial(functools.reduce, lambda x, y: x * y),
    )
)

### Что дальше?

1) Есть еще more-itertools, предоставляющие уйму других возможностей: https://more-itertools.readthedocs.io/en/stable/
2) Замечательная статья как упарываться декораторами и итераторами: https://www.bbayles.com/index/decorator_factory